In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.optimize import linear_sum_assignment
# import untils  # Provided utility
# from COBWEBNN import CobwebNN, CobwebNNTreeLayer, TestModel
from torchvision import datasets, transforms
from resnet_encoder import Encoder, BasicBlockEnc
from restnet_decoder import Decoder, BasicBlockDec
from resnet_using_light_basic_block_encoder import LightEncoder, LightBasicBlockEnc
from resnet_using_light_basic_block_decoder import LightDecoder, LightBasicBlockDec
import matplotlib.pyplot as plt
import numpy as np
# from classes.resnet_autoencoder import AE

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# test cuda
x = torch.tensor([1, 2, 3]).to(device)

In [5]:
download = True
dataset_class = datasets.MNIST
mnist_transform = [transforms.ToTensor()]
# add normalization to the transform list such that all values are strictly greater than 0
# mnist_transform.append(transforms.Lambda(lambda x: x + 1e-5))
# normalize the data
# mnist_transform.append(transforms.Normalize((0.1307,), (0.3081,)))

dataset_transform = transforms.Compose(mnist_transform)
mnist_train = dataset_class('data/MNIST', train=True, download=download, transform=dataset_transform)
mnist_test = dataset_class('data/MNIST', train=False, download=download, transform=dataset_transform)

mnist_train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)

In [15]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        # self.encoder = Encoder(BasicBlockEnc, [2, 2, 2, 2])
        # self.decoder = Decoder(BasicBlockDec, [2, 2, 2, 2])
        self.encoder = LightEncoder(LightBasicBlockEnc, [2, 2, 2]) 
        # projection_dim = 32
        self.n_hidden = 32
        self.projection = nn.Sequential(
            nn.Linear(64*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, self.n_hidden),
            # nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(self.n_hidden, 512),
            nn.ReLU(),
            nn.Linear(512, 64*7*7),
            nn.Unflatten(1, (64, 7, 7)),
            LightDecoder(LightBasicBlockDec, [2, 2, 2]),
        )

        # self.fc_encoder = nn.Linear(64*8*8, 512)
        # self.fc_decoder = nn.Linear(512, 64*8*8)
        # decoder MLP
        # self.decoder = nn.Sequential(
        #     nn.Linear(512, 64*2*2),
        #     nn.ReLU(True),
        #     nn.Linear(64*2*2, 64*4*4),
        #     nn.ReLU(True),
        #     nn.Linear(64*4*4, 3*32*32),
        #     nn.ReLU(True)
        # )

        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.ConTrans2d = nn.ConvTranspose2d(64, 64, 8, 8)

    def forward(self, x):
        x = self.encoder(x)
        z = self.projection(x.view(x.size(0), -1))
        # print(x.shape)
        
        # x = self.avgpool(x).view(x.size(0), -1)
        # x = x.view(x.size(0), -1)
        # x = self.fc_encoder(x)
        # print(x.shape)
        # x = self.fc_decoder(x).view(x.size(0), 64, 8, 8)

        # x = self.ConTrans2d(x)
        # print(x.shape)
        # x = self.fc_decoder(x)
        # x = x.view(x.size(0), 64, 8, 8)
        # z = self.de_projection(z)
        x = self.decoder(z)
        # print(x.shape)
        return x

In [16]:
def visualize_reconstruction(model, test_data, n_data=64, device='cuda', epochs=100):
    model.eval()
    test_loader = DataLoader(test_data, batch_size=n_data, shuffle=True)
    with torch.no_grad():
        for i, (x, _) in enumerate(test_loader):
            x = x.to(device)
            x = x.expand(-1, 3, -1, -1)
            x_recon = model(x)
            break
    model.train()
    # untils.visualize(x, x_recon, n_data)
    # 8*8 subplots
    fig, axs = plt.subplots(8, 8, figsize=(16, 16))
    for i in range(8):
        for j in range(8):
            axs[i, j].imshow(x_recon[i*8+j].cpu().permute(1, 2, 0))
            axs[i, j].axis('off')
    # plt.show()
    # save image to file
    plt.savefig(f'./ae_recon.png')
    plt.close()
    

In [ ]:
# train and test the model
model = AE().to(device)
# .to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

n_epochs = 10
loss_fn = nn.MSELoss()
for epoch in range(n_epochs):
    model.train()
    for i, (x, _) in enumerate(tqdm(mnist_train_loader)):
        x = x.to(device)
        # expand it as a 3 channel image
        x = x.expand(-1, 3, -1, -1)
        # print(x.min(), x.max())
        optimizer.zero_grad()
        x_recon = model(x)
        # print(x_recon.min(), x_recon.max())
        # break
        loss = loss_fn(x_recon, x)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    if epoch % 10 == 0:
        visualize_reconstruction(model, mnist_test, 64, device, epoch)

100%|██████████| 469/469 [00:12<00:00, 38.21it/s]


Epoch 0, Loss: 0.009492166340351105


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.1078343].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.1404964].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.1282963].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.2712901].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.1582433].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.1387092].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.2028837].
Clipping input data to the valid r

Epoch 1, Loss: 0.00533831212669611


 61%|██████    | 285/469 [00:07<00:04, 39.12it/s]